# 🪪 ID Card OCR — Arabic & French (PDF Input)

Handles a **scanned PDF** where:
- **Page 1** = Front of the ID card
- **Page 2** = Back of the ID card

Special feature: detects the **card serial number** (the first sequence of digits printed at the very top of the card) separately from the national ID number.

Pipeline:
1. Install dependencies
2. Extract PDF pages → images
3. Pre-process each page
4. Run bilingual OCR (Arabic + French)
5. Detect **card serial number** by position (top 20% of card)
6. Parse all other fields
7. Visualise both sides
8. Export unified JSON

## 📦 Cell 1 — Install Dependencies

In [ ]:
!pip install paddlepaddle paddleocr opencv-python-headless pillow \
             arabic-reshaper python-bidi matplotlib \
             pymupdf -q
# pymupdf (import fitz) — fast, accurate PDF → image conversion

## 📚 Cell 2 — Imports

In [ ]:
import cv2
import re
import json
import fitz                          # PyMuPDF
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from paddleocr import PaddleOCR
import arabic_reshaper
from bidi.algorithm import get_display

print('✅ All imports successful')

## ⚙️ Cell 3 — Configuration

In [ ]:
# ─── USER CONFIG ─────────────────────────────────────────────────────────────
PDF_PATH             = "id_card.pdf"       # ← your scanned PDF
FRONT_PAGE_INDEX     = 0                   # 0 = first page  (front of card)
BACK_PAGE_INDEX      = 1                   # 1 = second page (back of card)
PDF_DPI              = 300                 # higher = better quality, slower
CONFIDENCE_THRESHOLD = 0.55
USE_GPU              = False
OUTPUT_JSON          = "id_card_result.json"

# ─── Card serial number detection ─────────────────────────────────────────────
# The card serial / document number is the FIRST numeric sequence visible
# at the TOP of the card.  We detect it by:
#   1. Looking only in the top region of the image (TOP_REGION_FRACTION)
#   2. Matching a run of digits (SERIAL_PATTERN)
TOP_REGION_FRACTION  = 0.20               # top 20 % of the card height
SERIAL_PATTERN       = re.compile(r'\b(\d{4,})\b')  # ≥4 consecutive digits

# ─── Bilingual field keywords ─────────────────────────────────────────────────
FIELD_KEYWORDS = {
    "last_name": [
        "nom", "nom de famille", "surname",
        "الاسم العائلي", "اسم العائلة", "النسب",
    ],
    "first_name": [
        "prénom", "prenom", "given name",
        "الاسم الشخصي", "الاسم",
    ],
    "dob": [
        "date de naissance", "né le", "née le", "naissance",
        "تاريخ الازدياد", "تاريخ الميلاد",
    ],
    "pob": [
        "lieu de naissance", "né à", "née à",
        "مكان الازدياد", "مكان الميلاد",
    ],
    "expiry": [
        "valable jusqu'au", "expire le", "date d'expiration", "valid until",
        "صالحة إلى", "تاريخ الانتهاء",
    ],
    "id_number": [
        "n° carte", "numéro", "n°", "cin", "id no",
        "رقم البطاقة", "رقم التعريف الوطني",
    ],
    "address": [
        "adresse", "domicile", "résidence",
        "العنوان", "محل السكنى",
    ],
    "nationality": [
        "nationalité", "nationalite",
        "الجنسية",
    ],
    "sex": [
        "sexe", "genre",
        "الجنس",
    ],
}

DATE_PATTERN = re.compile(
    r'\b(\d{1,2}[\/-]\d{1,2}[\/-]\d{2,4}|\d{4}[\/-]\d{2}[\/-]\d{2})\b'
)
# National ID number — adjust regex to your country
# Morocco CIN: 1-2 letters + 5-6 digits | Algeria NNI: 18 digits
ID_PATTERN = re.compile(r'\b([A-Z]{1,2}\d{5,6}|\d{9,18})\b')
MRZ_PATTERN = re.compile(r'^[A-Z0-9<]{30,}$')

print('✅ Configuration loaded')

## 📄 Cell 4 — PDF → Images (both pages)

In [ ]:
def pdf_page_to_image(pdf_path: str, page_index: int, dpi: int = 300) -> np.ndarray:
    """
    Render a single PDF page to a BGR numpy array using PyMuPDF.
    DPI 300 is recommended for scanned documents.
    """
    doc  = fitz.open(pdf_path)
    if page_index >= len(doc):
        raise IndexError(
            f"PDF has only {len(doc)} page(s); requested page index {page_index}."
        )
    page = doc[page_index]
    zoom = dpi / 72                        # PyMuPDF default is 72 dpi
    mat  = fitz.Matrix(zoom, zoom)
    pix  = page.get_pixmap(matrix=mat, alpha=False)
    img  = np.frombuffer(pix.samples, dtype=np.uint8)
    img  = img.reshape(pix.height, pix.width, 3)   # RGB
    return cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


def show_side_by_side(img1: np.ndarray, img2: np.ndarray,
                      title1="Front", title2="Back"):
    fig, axes = plt.subplots(1, 2, figsize=(18, 10))
    for ax, img, title in [(axes[0], img1, title1), (axes[1], img2, title2)]:
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_title(title, fontsize=14)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


print(f'⏳ Extracting pages from {PDF_PATH} at {PDF_DPI} DPI...')
front_raw = pdf_page_to_image(PDF_PATH, FRONT_PAGE_INDEX, PDF_DPI)
back_raw  = pdf_page_to_image(PDF_PATH, BACK_PAGE_INDEX,  PDF_DPI)

print(f'   Front page shape : {front_raw.shape}')
print(f'   Back  page shape : {back_raw.shape}')
show_side_by_side(front_raw, back_raw, '📄 Front (raw)', '📄 Back (raw)')

## 🔧 Cell 5 — Pre-processing

In [ ]:
def resize_if_small(img: np.ndarray, min_side: int = 1000) -> np.ndarray:
    h, w = img.shape[:2]
    scale = max(min_side / min(h, w), 1.0)
    if scale > 1.0:
        img = cv2.resize(img, (int(w * scale), int(h * scale)),
                         interpolation=cv2.INTER_CUBIC)
    return img

def denoise(img):
    return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

def deskew(img: np.ndarray) -> np.ndarray:
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180,
                             threshold=80, minLineLength=80, maxLineGap=10)
    if lines is None:
        return img
    angles = [np.degrees(np.arctan2(y2-y1, x2-x1))
               for x1,y1,x2,y2 in lines[:,0]
               if abs(np.degrees(np.arctan2(y2-y1, x2-x1))) < 45]
    if not angles or abs(np.median(angles)) < 0.5:
        return img
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), np.median(angles), 1.0)
    return cv2.warpAffine(img, M, (w, h),
                          flags=cv2.INTER_CUBIC,
                          borderMode=cv2.BORDER_REPLICATE)

def enhance_contrast(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return cv2.cvtColor(cv2.merge([clahe.apply(l), a, b]), cv2.COLOR_LAB2BGR)

def sharpen(img):
    k = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
    return cv2.filter2D(img, -1, k)

def preprocess(img: np.ndarray) -> np.ndarray:
    img = resize_if_small(img)
    img = denoise(img)
    img = deskew(img)
    img = enhance_contrast(img)
    img = sharpen(img)
    return img


print('⏳ Pre-processing front page...')
front = preprocess(front_raw)
print('⏳ Pre-processing back page...')
back  = preprocess(back_raw)

show_side_by_side(front, back, '✅ Front (processed)', '✅ Back (processed)')

## 🔤 Cell 6 — Initialise OCR Engines

In [ ]:
print('⏳ Loading Arabic OCR engine...')
ocr_ar = PaddleOCR(use_angle_cls=True, lang='ar', use_gpu=USE_GPU, show_log=False)
print('✅ Arabic engine ready')

print('⏳ Loading French / Latin OCR engine...')
ocr_fr = PaddleOCR(use_angle_cls=True, lang='fr', use_gpu=USE_GPU, show_log=False)
print('✅ French engine ready')

## 🔍 Cell 7 — OCR Helper Functions

In [ ]:
def is_arabic(text: str) -> bool:
    return bool(re.search(r'[\u0600-\u06FF]', text))

def render_arabic(text: str) -> str:
    if is_arabic(text):
        return get_display(arabic_reshaper.reshape(text))
    return text

def extract_lines(ocr_engine, img: np.ndarray, language: str) -> list:
    """Run OCR and return list of dicts with text, confidence, bbox, position info."""
    h, w = img.shape[:2]
    results = ocr_engine.ocr(img, cls=True)
    lines = []
    for page in results:
        if not page:
            continue
        for item in page:
            bbox, (text, conf) = item
            text = text.strip()
            if not text or conf < CONFIDENCE_THRESHOLD:
                continue
            pts = np.array(bbox)
            cx  = float(pts[:, 0].mean())
            cy  = float(pts[:, 1].mean())
            # Normalised vertical position  0 = top, 1 = bottom
            y_norm = cy / h
            lines.append({
                'text':       text,
                'confidence': round(float(conf), 4),
                'bbox':       bbox,
                'language':   language,
                'cx': cx, 'cy': cy,
                'y_norm': round(y_norm, 4),    # ← used for serial detection
            })
    return lines


def iou(b1, b2) -> float:
    def to_rect(b):
        pts = np.array(b); x1,y1 = pts.min(axis=0); x2,y2 = pts.max(axis=0)
        return x1, y1, x2, y2
    ax1,ay1,ax2,ay2 = to_rect(b1)
    bx1,by1,bx2,by2 = to_rect(b2)
    ix1,iy1 = max(ax1,bx1), max(ay1,by1)
    ix2,iy2 = min(ax2,bx2), min(ay2,by2)
    inter = max(0, ix2-ix1) * max(0, iy2-iy1)
    union = (ax2-ax1)*(ay2-ay1) + (bx2-bx1)*(by2-by1) - inter
    return inter/union if union > 0 else 0.0


def deduplicate(ar_lines: list, fr_lines: list) -> list:
    """Merge Arabic + French results; resolve overlaps by script and confidence."""
    merged = list(ar_lines)
    for fr in fr_lines:
        overlap = False
        for ar in ar_lines:
            if iou(fr['bbox'], ar['bbox']) > 0.4:
                overlap = True
                # Both Latin → keep higher confidence
                if not is_arabic(ar['text']) and not is_arabic(fr['text']):
                    if fr['confidence'] > ar['confidence']:
                        merged = [fr if x is ar else x for x in merged]
                break
        if not overlap:
            merged.append(fr)
    # Reading order: top → bottom, left → right
    merged.sort(key=lambda x: (round(x['cy'] / 30), x['cx']))
    return merged


def run_ocr_on_page(img: np.ndarray, label: str = '') -> list:
    print(f'  Running Arabic OCR {label}...')
    ar = extract_lines(ocr_ar, img, 'ar')
    print(f'    → {len(ar)} Arabic detections')
    print(f'  Running French OCR {label}...')
    fr = extract_lines(ocr_fr, img, 'fr')
    print(f'    → {len(fr)} French detections')
    merged = deduplicate(ar, fr)
    print(f'  Merged: {len(merged)} unique detections')
    return merged

print('✅ Helper functions defined')

## 🔍 Cell 8 — Run OCR on Both Pages

In [ ]:
print('\n── FRONT PAGE ──────────────────────────────────')
front_lines = run_ocr_on_page(front, '(front)')

print('\n── BACK PAGE ───────────────────────────────────')
back_lines  = run_ocr_on_page(back,  '(back)')

## 🔢 Cell 9 — Card Serial Number Detection

The **card serial / document number** is the first sequence of digits printed at the **very top** of the card.  
Strategy: look only at detections whose vertical centre is within the top `TOP_REGION_FRACTION` of the image, then pick the **leftmost** numeric sequence found there.

In [ ]:
def detect_card_serial(lines: list,
                       top_fraction: float = TOP_REGION_FRACTION) -> dict | None:
    """
    Identify the card serial number from OCR lines.

    Algorithm:
    1. Filter lines whose vertical centre is in the top `top_fraction` of the card.
    2. Among those, find every token that is a run of ≥4 digits.
    3. Sort candidates by their horizontal position (left → right).
    4. Return the FIRST (leftmost / topmost) candidate.

    Returns a dict {serial, confidence, bbox, y_norm} or None.
    """
    candidates = []

    for item in lines:
        if item['y_norm'] > top_fraction:        # outside top band → skip
            continue
        for m in SERIAL_PATTERN.finditer(item['text']):
            candidates.append({
                'serial':     m.group(1),
                'confidence': item['confidence'],
                'bbox':       item['bbox'],
                'y_norm':     item['y_norm'],
                'cx':         item['cx'],
                'cy':         item['cy'],
                'full_text':  item['text'],
            })

    if not candidates:
        return None

    # Sort by (vertical position, then horizontal) → topmost-left wins
    candidates.sort(key=lambda c: (c['y_norm'], c['cx']))
    return candidates[0]


def visualise_top_region(img: np.ndarray, lines: list,
                          serial_result: dict | None,
                          top_fraction: float = TOP_REGION_FRACTION,
                          title: str = ''):
    """Show the card with the top-region band and the detected serial highlighted."""
    vis  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).copy()
    h, w = img.shape[:2]
    band = int(h * top_fraction)

    # Semi-transparent yellow band over the top region
    overlay = vis.copy()
    cv2.rectangle(overlay, (0, 0), (w, band), (255, 230, 50), -1)
    vis = cv2.addWeighted(overlay, 0.25, vis, 0.75, 0)

    # Mark all detections in the top band
    for item in lines:
        if item['y_norm'] <= top_fraction:
            pts = np.array(item['bbox'], np.int32)
            cv2.polylines(vis, [pts], True, (180, 180, 0), 2)

    # Highlight the chosen serial box in green
    if serial_result:
        pts = np.array(serial_result['bbox'], np.int32)
        cv2.polylines(vis, [pts], True, (0, 220, 80), 4)
        x, y = pts[0]
        cv2.putText(vis, f"SERIAL: {serial_result['serial']}",
                    (x, max(y - 12, 0)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 200, 60), 2)

    plt.figure(figsize=(14, 8))
    plt.imshow(vis)
    plt.title(title or 'Serial Number Detection', fontsize=13)
    plt.axis('off')
    plt.tight_layout()
    plt.show()


# ── Run on front page (serial is usually on the front) ──────────────────────
serial_result = detect_card_serial(front_lines)

print('\n🔢 Card Serial Number Detection')
print('─' * 40)
if serial_result:
    print(f"  Serial number  : {serial_result['serial']}")
    print(f"  Full text line : {serial_result['full_text']}")
    print(f"  Confidence     : {serial_result['confidence']:.2f}")
    print(f"  Vertical pos   : top {serial_result['y_norm']*100:.1f}% of card")
else:
    print('  ⚠️  No serial number found in the top region.')
    print('  → Try increasing TOP_REGION_FRACTION in Cell 3.')

visualise_top_region(
    front, front_lines, serial_result,
    title=f"🔢 Serial Detection — top {TOP_REGION_FRACTION*100:.0f}% region (yellow)"
)

## 🗂️ Cell 10 — Parse All Fields (Both Sides)

In [ ]:
def parse_id_fields(lines: list) -> dict:
    """Extract structured fields from merged OCR lines."""
    fields = {k: None for k in FIELD_KEYWORDS}
    fields['mrz']       = []
    fields['all_dates'] = []
    fields['all_text']  = [i['text'] for i in lines]

    for i, item in enumerate(lines):
        text  = item['text']
        lower = text.lower()

        # MRZ line
        if MRZ_PATTERN.match(text):
            fields['mrz'].append(text)
            continue

        # Dates
        for m in DATE_PATTERN.finditer(text):
            fields['all_dates'].append(m.group())

        # National ID number
        if not fields['id_number']:
            m = ID_PATTERN.search(text)
            if m:
                fields['id_number'] = m.group(1)

        # Keyword-based fields
        for field, keywords in FIELD_KEYWORDS.items():
            if fields[field]:
                continue
            if any(kw in lower or kw in text for kw in keywords):
                if ':' in text:
                    value = text.split(':', 1)[-1].strip()
                elif '،' in text:
                    value = text.split('،', 1)[-1].strip()
                elif i + 1 < len(lines):
                    value = lines[i + 1]['text']
                else:
                    value = text
                if value:
                    fields[field] = value

    # Assign dates heuristically
    dates = fields['all_dates']
    if dates:
        if not fields['dob']:    fields['dob']    = dates[0]
        if not fields['expiry'] and len(dates) >= 2:
            fields['expiry'] = dates[-1]

    return fields


# Merge both sides for field parsing (front is primary)
all_lines = front_lines + back_lines
parsed    = parse_id_fields(all_lines)

# Inject card serial detected in Cell 9
parsed['card_serial'] = serial_result['serial'] if serial_result else None

print('\n' + '═'*52)
print('         🪪  PARSED ID CARD FIELDS')
print('═'*52)
DISPLAY_ORDER = [
    'card_serial', 'id_number', 'last_name', 'first_name',
    'dob', 'pob', 'sex', 'nationality', 'address', 'expiry'
]
for field in DISPLAY_ORDER:
    value = parsed.get(field)
    display_val = render_arabic(str(value)) if value else '—'
    marker = '🆔' if field == 'card_serial' else '  '
    print(f'{marker} {field:<16}: {display_val}')

if parsed['all_dates']:
    print(f'\n  All dates found : {parsed["all_dates"]}')
if parsed['mrz']:
    print('\n  MRZ Lines:')
    for line in parsed['mrz']:
        print(f'    {line}')

## 🎨 Cell 11 — Visualise OCR Boxes on Both Pages

In [ ]:
COLOUR_MAP = {'ar': (255, 80, 80), 'fr': (50, 180, 255)}

def annotate_page(img_bgr: np.ndarray, lines: list,
                  serial: dict | None = None) -> np.ndarray:
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)
    draw    = ImageDraw.Draw(pil_img)
    try:
        font = ImageFont.truetype(
            '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', 20)
    except Exception:
        font = ImageFont.load_default()

    for item in lines:
        pts    = [tuple(map(int, pt)) for pt in item['bbox']]
        colour = COLOUR_MAP.get(item['language'], (0, 220, 0))
        for j in range(4):
            draw.line([pts[j], pts[(j+1)%4]], fill=colour, width=2)
        label = render_arabic(item['text'])[:40]
        x, y  = pts[0]
        draw.text((x, max(y-22, 0)), label, fill=colour, font=font)

    # Highlight serial box
    if serial:
        pts = [tuple(map(int, pt)) for pt in serial['bbox']]
        for j in range(4):
            draw.line([pts[j], pts[(j+1)%4]], fill=(0, 220, 80), width=4)
        x, y = pts[0]
        draw.text((x, max(y-26, 0)),
                  f"▶ SERIAL: {serial['serial']}",
                  fill=(0, 200, 60), font=font)

    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)


ann_front = annotate_page(front, front_lines, serial=serial_result)
ann_back  = annotate_page(back,  back_lines)

ar_patch = mpatches.Patch(color=(1.0, 0.31, 0.31), label='Arabic')
fr_patch = mpatches.Patch(color=(0.20, 0.71, 1.0),  label='French')
sn_patch = mpatches.Patch(color=(0.0,  0.86, 0.47), label='Card serial')

fig, axes = plt.subplots(1, 2, figsize=(20, 11))
for ax, img, title in [
    (axes[0], ann_front, '🪪 Front — OCR annotations'),
    (axes[1], ann_back,  '🪪 Back — OCR annotations'),
]:
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax.set_title(title, fontsize=13)
    ax.axis('off')

fig.legend(handles=[ar_patch, fr_patch, sn_patch],
           loc='lower center', ncol=3, fontsize=12)
plt.tight_layout()
plt.show()

## 📊 Cell 12 — Confidence & Position Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

# Confidence histograms
for ax, lang, colour, label in [
    (axes[0], 'ar', '#FF5050', 'Arabic'),
    (axes[1], 'fr', '#32B4FF', 'French'),
]:
    confs = [l['confidence'] for l in all_lines if l['language'] == lang]
    ax.hist(confs or [0], bins=20, range=(0,1), color=colour,
            edgecolor='white', alpha=0.85)
    if confs:
        ax.axvline(np.mean(confs), color='black', linestyle='--',
                   label=f'Mean: {np.mean(confs):.2f}')
    ax.set_title(f'{label} Confidence', fontsize=12)
    ax.set_xlabel('Confidence'); ax.set_ylabel('Count')
    ax.legend()

# Vertical distribution (where on the card detections appear)
y_norms = [l['y_norm'] for l in front_lines]
axes[2].hist(y_norms, bins=20, range=(0,1), color='#9B59B6',
             edgecolor='white', alpha=0.85)
axes[2].axvline(TOP_REGION_FRACTION, color='orange', linestyle='--',
                linewidth=2, label=f'Top region ({TOP_REGION_FRACTION*100:.0f}%)')
axes[2].set_title('Front: Vertical Position of Detections', fontsize=12)
axes[2].set_xlabel('Normalised Y (0=top, 1=bottom)')
axes[2].set_ylabel('Count')
axes[2].legend()

plt.suptitle('OCR Analysis', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 💾 Cell 13 — Export to JSON

In [ ]:
def make_serialisable(lines: list) -> list:
    out = []
    for item in lines:
        out.append({
            **{k: v for k, v in item.items() if k not in ('bbox',)},
            'bbox': [[float(x), float(y)] for x, y in item['bbox']],
        })
    return out


output = {
    'source_pdf': PDF_PATH,
    'parsed_fields': {
        k: v for k, v in parsed.items() if k != 'all_text'
    },
    'card_serial_detail': serial_result and {
        'serial':     serial_result['serial'],
        'confidence': serial_result['confidence'],
        'y_norm':     serial_result['y_norm'],
        'full_text':  serial_result['full_text'],
        'bbox':       [[float(x), float(y)] for x, y in serial_result['bbox']],
    },
    'front_lines': make_serialisable(front_lines),
    'back_lines':  make_serialisable(back_lines),
}

with open(OUTPUT_JSON, 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print(f'✅ Results saved → {OUTPUT_JSON}')
print(f'   Front detections : {len(front_lines)}')
print(f'   Back  detections : {len(back_lines)}')
print(f'   Card serial      : {parsed.get("card_serial")}')
print(f'   ID number        : {parsed.get("id_number")}')

## 🔁 Cell 14 — Tuning the Serial Number Detector

If the serial number was **not detected**, run this cell to explore what was found in the top region and adjust parameters.

In [ ]:
# ── Diagnostic: what OCR found in the top portion of the front page ──────────
PROBE_FRACTION = 0.30   # ← increase this to look lower on the card

print(f'Detections in top {PROBE_FRACTION*100:.0f}% of front page:')
print(f'{"Y%":>5}  {"Conf":>5}  {"Lang":>4}  Text')
print('-' * 65)

top_items = [l for l in front_lines if l['y_norm'] <= PROBE_FRACTION]
top_items.sort(key=lambda x: (x['y_norm'], x['cx']))

for item in top_items:
    print(f"{item['y_norm']*100:>4.1f}%  {item['confidence']:>5.2f}  "
          f"{item['language']:>4}  {render_arabic(item['text'])}")

if not top_items:
    print('  Nothing found. Try lowering CONFIDENCE_THRESHOLD or raising PROBE_FRACTION.')

---
## 📝 Reference

| Parameter | Where | Effect |
|---|---|---|
| `TOP_REGION_FRACTION` | Cell 3 | Fraction of card height scanned for serial number |
| `SERIAL_PATTERN` | Cell 3 | Regex matching the serial format (default: ≥4 digits) |
| `CONFIDENCE_THRESHOLD` | Cell 3 | Drop OCR results below this score |
| `PDF_DPI` | Cell 3 | Rendering resolution — 300 recommended, 400 for poor scans |
| `FRONT_PAGE_INDEX` | Cell 3 | 0-based page index for the front of the card |
| `BACK_PAGE_INDEX` | Cell 3 | 0-based page index for the back of the card |
| `ID_PATTERN` | Cell 3 | Regex for the national ID number format |

**Difference between `card_serial` and `id_number`:**
- `card_serial` — the document/card serial printed at the **physical top** of the card, detected purely by **position** (topmost numeric string).
- `id_number` — the **national identity number** found by matching keywords like `CIN`, `رقم البطاقة`, etc., or by pattern (letter + digits).